# Assignment 2


You must submit your notebook by running `python3 -m autograder.run.submit Assignment2.ipynb` from your local repository.

To write legible answers you will need to be familiar with both [Markdown](https://github.com/adam-p/markdown-here/wiki/Markdown-Cheatsheet) and [Latex](https://www.latex-tutorial.com/tutorials/amsmath/)

Before you turn this problem in, make sure everything runs as expected. To do so, restart the kernel and run all cells (in the menubar, select Runtime→→Restart and run all).

#### Show your work!
Whenever you are asked to find the solution to a problem, be sure to also **show how you arrived** at your answer.

Make sure you fill in any place that says "YOUR CODE HERE" or "YOUR ANSWERS HERE", as well as your name below:

## Some Helper Functions

Implement an evaluation function that takes in some board position and player colour and returns a score. 

In [104]:
import chess
import random
from math import inf
from IPython.display import display, clear_output

In [117]:
def evaluation(b: chess.Board, player: bool):
    """
    This function evaluates a board position and returns a score value.

    Parameters:
    - board: the chess board that the knight is moving upon
    - player: the colour of the active player (True -> white, False -> black)

    Returns:
    - an integer score for a board state
    """
    b = b.copy()
    weight_lookup = {
        chess.PAWN: 1,
        chess.KNIGHT: 3,
        chess.BISHOP: 3,
        chess.ROOK: 5,
        chess.QUEEN: 9,
        chess.KING: 200
    }
    material_score = 0
    player_color = chess.WHITE if player else chess.BLACK
    opponent_color = chess.BLACK if player else chess.WHITE
    # material score
    for piece_type in weight_lookup:
        player_pieces = len(b.pieces(piece_type, player_color))
        opponent_pieces = len(b.pieces(piece_type, opponent_color))
        material_score += weight_lookup[piece_type] * (player_pieces - opponent_pieces)

    # mobility score
    player_moves = list(b.legal_moves)
    b.turn = opponent_color
    opponent_moves = list(b.legal_moves)
    mobility_score = 0.1 * (len(player_moves) - len(opponent_moves))
    # could consider adding score for doubled, blocked, and isolated pawns

    aggressive_score = 0

    if b.is_check():
        aggressive_score += 3
    if b.is_checkmate():
        aggressive_score += 100

    score = (material_score + mobility_score + aggressive_score) * (1 if player else -1)
    return score


This evaluation logic is found from https://www.chessprogramming.org/Evaluation

## Q1

Implement the minimax algorithm to choose the best chess move for a given board position, player colour, and look-ahead depth. Make use of your `evaluation()` function here to calculate the score of any individual board state.

In [106]:
def get_minimax_move(b: chess.Board, player: bool, depth: int):
    """
    This function chooses the best move for the given board position, player, and depth.

    Parameters:
    - board: the chess board that the knight is moving upon
    - player: the colour of the active player (True -> white, False -> black)
    - depth: the number of moves that the algorithmn should look ahead.

    Returns:
    A single chess.Move type object.
    """
    b = b.copy()
    def helper(local_b: chess.Board, local_player: bool, local_depth: int, alpha, beta) -> tuple:
        """ returns (score, move) """
        local_b = local_b.copy()
        if local_depth == 0 or local_b.is_game_over():
            return evaluation(local_b, player)

        legal_moves = local_b.legal_moves
        if local_player:
            # max node
            v = -inf
            for move in legal_moves:
                local_b.push(move)
                next_score = helper(local_b, not local_player, local_depth - 1, alpha, beta)
                v = max(v, next_score)
                local_b.pop()
                alpha = max(alpha, v)
                if alpha >= beta:
                    break
            return v

        else:
            # min node
            v = inf
            for move in legal_moves:
                local_b.push(move)
                next_score = helper(local_b, not local_player, local_depth - 1, alpha, beta)
                v = min(v, next_score)
                local_b.pop()
                beta = min(beta, v)
                if alpha >= beta:
                    break
            return v

    best_move = None
    max_score = -inf
    legal_moves = b.legal_moves
    for move in legal_moves:
        b.push(move)
        score = helper(b, player, depth, -inf, inf)
        b.pop()
        if score > max_score:
            max_score = score
            best_move = move

    return best_move

## Q2

Implement the expectimax algorithm to choose the best chess move for a given board position, player colour, and look-ahead depth. Make use of your `evaluation()` function here to calculate the score of any individual board state.

In [107]:
def get_expectimax_move(b: chess.Board, player: bool, depth: int):
    """
    This function chooses the best move for the given board position, player, and depth.

    Parameters:
    - board: the chess board that the knight is moving upon
    - player: the colour of the active player (True -> white, False -> black)
    - depth: the number of moves that the algorithmn should look ahead.

    Returns:
    A single chess.Move type object.
    """
    b = b.copy()

    def helper(local_b: chess.Board, local_player: bool, local_depth: int) -> tuple:
        """ returns (score, move) """
        local_b = local_b.copy()
        if local_depth == 0 or local_b.is_game_over():
            return evaluation(local_b, player)

        legal_moves = local_b.legal_moves
        total_legal_moves = len(list(legal_moves))

        uniform_probability = 1 / total_legal_moves
        if local_player:
            # max node
            v = -inf
            for move in legal_moves:
                local_b.push(move)
                eval = helper(local_b, not local_player, local_depth - 1)
                local_b.pop()
                v = max(v, eval)
            return v

        else:
            # exp node
            v = 0
            for move in legal_moves:
                local_b.push(move)
                eval = helper(local_b, not local_player, local_depth - 1)
                local_b.pop()
                v += eval
            return v / total_legal_moves if total_legal_moves > 0 else 0

    best_move = None
    max_score = -inf
    legal_moves = b.legal_moves
    for move in legal_moves:
        b.push(move)
        score = helper(b, player, depth)
        b.pop()
        if score > max_score:
            max_score = score
            best_move = move

    return best_move

## Q3

Plays against the strongest grading agent, so copy your best agent here.

In [121]:
def get_best_move(b: chess.Board, player: bool, depth: int):
    """
    This function chooses the best move for the given board position, player, and depth.

    Parameters:
    - board: the chess board that the knight is moving upon
    - player: the colour of the active player (True -> white, False -> black)
    - depth: the number of moves that the algorithmn should look ahead.

    Returns:
    A single chess.Move type object.
    """

    # p = b.piece_at(chess.E2)
    # opening = False
    # if p and p.piece_type == chess.PAWN and p.color == chess.WHITE:
    #     opening = True
    # if player and opening:
    #     return chess.Move(chess.E2, chess.E4)
    # k = b.piece_at(chess.G1)
    # if k and k.piece_type == chess.KNIGHT and k.color == chess.WHITE:
    #     opening = True
    # if player and opening:
    #     return chess.Move(chess.G1, chess.F3)
    return get_minimax_move(b, player, depth)
    # return get_expectimax_move(b, player, depth)

## Local Testing
For your convenience, here are some helper functions to run games and allow you to test your agents.

In [109]:
MAX_DEPTH = 3
SEED = random.Random(0)

def get_random_move(b:chess.Board, *_):
    return SEED.choice(list(b.legal_moves))

def test_game(p1, p2):
    """
    A function that plays a chess game with visuals

    Parameters:
    - p1: the first chess agent, playing as the white player
    - p2: the second chess agent, playing as the black player

    Returns:
    Nothing, just plays the game one move at a time.
    Press enter in the popup box to play the next move.
    Type 'q' into the popup box and press enter to stop the game (you can also interrupt the kernel if this fails.)
    """
    global SEED
    SEED = random.Random(0)
    board = chess.Board()
    current_player = p1

    while not board.is_checkmate()  and not board.is_stalemate() and not board.can_claim_draw():
        clear_output(True)

        move = current_player(board, board.turn, MAX_DEPTH)
        board.push(move)
        display(board)

        current_player = p1 if board.turn else p2

        if input() == "q":
            break


In [110]:
# Random Vs. Random
random_agent = get_random_move
# test_game(random_agent, random_agent)

In [111]:
# Minimax Vs. Random
random_agent = get_random_move
minimax_agent = get_minimax_move
# test_game(minimax_agent, random_agent)


In [112]:
# Expectimax Vs. Random
random_agent = get_random_move
expectimax_agent = get_expectimax_move
# test_game(expectimax_agent, random_agent)

In [113]:
# Run several games at once without graphics
def grade_game(p1, p2):
    """
    Grades a single game at a time.
    (Note: differs from actual grading script, which has some ways to resolve draws.)

    Return legend:
    0: error during game
    1: p1 wins through checkmate
    2: p2 wins through checkmate
    3: draw
    """
    global SEED
    SEED = random.Random(0)
    board = chess.Board()
    current_player = p1

    while not board.is_checkmate() and not board.is_stalemate() and not board.can_claim_draw():
        move = current_player(board, MAX_DEPTH, board.turn)
        try:
            board.push(move)
        except:
            print(f"Error while grading game, move = {move}, current_player = {current_player}")
            print(board)
            return -1

        current_player = p1 if board.turn else p2

    outcome = board.outcome()
    if outcome is not None:
        if outcome.winner:
            return 1
        else:
            return 2

    return 3

In [122]:
# Play many games in a row without visuals (faster)
random_agent = get_random_move
best_agent = get_best_move
num_games = 5

print(f"Playing {num_games} games...")
for i in range(1, num_games+1):
    result = grade_game(best_agent, random_agent)

    if result == 0:
        print(f"Game {i}: error pushing a move during gameplay")

    elif result == 1:
        print(f"Game {i}: p1 wins")

    elif result == 2:
        print(f"Game {i}: p1 loses")

    elif result == 3:
        print(f"Game {i}: draw")

Playing 5 games...
Game 1: p1 wins
Game 2: p1 wins
Game 3: p1 wins
Game 4: p1 wins
Game 5: p1 wins
